# Locust Load Testing

## Machine requriements

This notebook should be run on a machine with at least 8 Nvidia GPUs, or 16 AWS AI chips. Below we show recommended machines for a range of LLM sizes. 

**Note:** As LLM size grows bigger, it may not fit on a single machine, and cannot be tested using this notebook.

### For Nvidia GPUs

| LLM size      | EC2 instance type |
| ----------- | ----------- |
| Upto 10B      | [g6.48xlarge](https://aws.amazon.com/ec2/instance-types/g6/), [g5.48xlarge](https://aws.amazon.com/ec2/instance-types/g5/)      |
| Upto 70B   | [g6e.48xlarge](https://aws.amazon.com/ec2/instance-types/g6e/), [p4d.24xlarge](https://aws.amazon.com/ec2/instance-types/p4/)      |
| Upto 405B   | [p5.48xlarge](https://aws.amazon.com/ec2/instance-types/p5/), [p4de.24xlarge](https://aws.amazon.com/ec2/instance-types/p4/)      |
| > 405B   | [p5e.48xlarge](https://aws.amazon.com/ec2/instance-types/p5/)      |

### For AWS AI Chips

| LLM size      | EC2 instance type |
| ----------- | ----------- |    
| Upto 70B   | [trn1.32xlarge](https://aws.amazon.com/ec2/instance-types/trn1/)       |
| > 70B   | [trn2.48xlarge](https://aws.amazon.com/ec2/instance-types/trn2/)      |

Next, let us verify the machine.


In [1]:
import subprocess
p = subprocess.run('nvidia-smi --list-gpus | wc -l', 
                   shell=True, check=True, capture_output=True, encoding='utf-8')


device = None
num_device = 0

if p.returncode == 0:
    num_device = int(p.stdout)
    device = "cuda" if num_device > 0 else None

if device is None:
    p = subprocess.run('neuron-ls -j | grep neuron_device | wc -l', 
                       shell=True, check=True, capture_output=True, encoding='utf-8')
    if p.returncode == 0:
        num_device = int(p.stdout)
        device = "neuron" if num_device > 0 else None

assert (device == "cuda" and num_device == 8) or (device == "neuron" and num_device == 16), \
    "Machine must have 8 Nvidia GPUs, or 16 AWS AI Chips"

Next, install required Python packages.

In [ ]:
!pip install --upgrade pip
!pip install locust
!pip install datasets
!which locust

### Create Hugging Face User Access Token

Many of the popular Large Language Models (LLMs) in Hugging Face are [gated models](https://huggingface.co/docs/hub/en/models-gated). To access gated models, you need a Hugging Face [user access token](https://huggingface.co/docs/hub/en/security-tokens). Please create a Hugging Face user access token in your Hugging Face account, and set it below in `hf_token` variable below.

In [ ]:
import subprocess
import time
import os
import stat
import yaml
import json

hf_token=''
# Comment out next line if not using a Hugging Face gated model
assert hf_token, "Hugging Face user access token is required for gated models"
os.environ['HF_TOKEN']=hf_token

### Build Docker Containers

Next, we build the docker containers used to run the inference endpoint locally on this desktop.


In [ ]:
! source build-containers.sh

### Specify Hugging Face Model Id

Next, set the Hugging Face Model Id for the LLM you want to test in `hf_model_id` variable, below. You can specify the Hugging Face model id for a [Meta Llama](https://huggingface.co/meta-llama), [Deepseek](https://huggingface.co/deepseek-ai), or [Mistral AI](https://huggingface.co/mistralai) text-based Generative AI LLM. Multi-modal LLMs are not yet supported in this notebook. 

In [ ]:
hf_model_id = 'meta-llama/Llama-3.2-1B-Instruct'
os.environ['MODEL_ID']=hf_model_id

### Specify Inference Server and Backend

Next, specify `inference_server`, and `backend` variables, below. This notebook supports [Triton Inference Server](https://github.com/triton-inference-server/server) with [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM), [vLLM](https://github.com/vllm-project/vllm), and [Transformers Neuronx in LMI engines](https://docs.djl.ai/master/docs/serving/serving/docs/lmi/user_guides/tnx_user_guide.html) as backends. 

In [ ]:
inference_server = 'triton_inference_server'
assert inference_server in ['triton_inference_server']
backend = 'vllm'
assert backend in ['vllm', 'trtllm', 'tnx_lmi']

assert backend != 'trtllm' or device == "cuda", f"TesnorRT-LLM not supported on {device} device"
assert backend != 'tnx_lmi' or device == "neuron", f"Transformers-Neuronx in LMI is not supported on {device} device"

### Launch Inference Server

Next we use Docker compose to launch the inference server locally on this desktop.

In [ ]:
script_map = {
    "triton_inference_server": {
        "vllm": {
            "cuda": "../triton-server/vllm/compose-triton-vllm-cuda.sh",
            "neuron": "../triton-server/vllm/compose-triton-vllm-neuronx.sh"
        },
        "trtllm": {
            "cuda": "../triton-server/tensorrt-llm/compose-triton-tensorrt-llm.sh"
        },
        "tnx_lmi": {
            "neuron": "../triton-server/djl-lmi/compose-triton-djl-lmi-neuronx.sh"
        }
    }
}

script_path = script_map[inference_server][backend][device]
! {script_path} down
! {script_path} up

## Locust Testing

### Load Configuration

Below. we load the appropriate configuration file for the specified inference server, and backend.

In [ ]:
config_path=f"config/{inference_server}-{backend}.yaml"
with open(config_path, "r") as mf:
    config=yaml.safe_load(mf)

print(json.dumps(config, indent=2))

### Verify inference server is up

The inference server may take several minutes to start up. Next, we verify the inference server is up. Do not proceed to next cell until inference server is up.

In [ ]:
import requests
try:
    response = requests.get(config['endpoint_url'])
    assert int(response.status_code) == 405, "Inference server is not yet up"
    print("Inference server is up!")
except:
    print("Inference server is not yet up")

### Validate Configuration

The configuration file specifies a prompt generator module. The module is dynamically loaded, and is invoked iteratively by the Locust endpoint user (see `endpoint_user.py`) to get next prompt to drive Locust testing.

Let us validate our configuration by making a single request and inspecting the response.

In [ ]:
from importlib import import_module
import sys
from pprint import pprint

def get_prompt_generator():
    prompt_module_dir = config['module_dir']
    sys.path.append(prompt_module_dir)
    
    prompt_module_name = config['module_name']
    prompt_module=import_module(prompt_module_name)
    
    prompt_generator_name = config['prompt_generator']
    prompt_generator_class = getattr(prompt_module, prompt_generator_name)

    kwargs = config.get('prompt_kwargs', {})
    return prompt_generator_class()(), kwargs
      
def inference_request():
    prompt_generator, kwargs = get_prompt_generator()
    text_input = next(prompt_generator)
    data= { "text_input": text_input }
    data.update(kwargs)
    
    body = json.dumps(data).encode("utf-8")
    pprint(body)
    headers = {"Content-Type":  "application/json"}
    response = requests.post(config['endpoint_url'], data=body, headers=headers)
    return response

response = inference_request()
assert int(response.status_code) == 200, f"Response status code {response.status_code} != 200"
pprint(f"Response Content: {response.json()}")

### Run Locust Load Testing

The Locust load testing below uses 32 users with 32 workers to drive concurrent load, and by default, is set to run for 60 seconds. You can adjust these values as needed. Keep `SPAWN_RATE` the same as `USERS` to drive maximum concurrency.

In [ ]:
ts = round(time.time() * 1000)

os.environ["PROMPT_MODULE_DIR"] = config['module_dir']
os.environ["PROMPT_MODULE_NAME"] = config['module_name']
os.environ["PROMPT_GENERATOR_NAME"] = config['prompt_generator']
os.environ["PROMPT_KWARGS"] = json.dumps(config.get('prompt_kwargs', {}))
os.environ["CONTENT_TYPE"]="application/json"
os.environ["ENDPOINT_NAME"] = config['endpoint_url']
os.environ["USERS"]="32"
os.environ["WORKERS"]="32"
os.environ["RUN_TIME"]="60s"
os.environ["SPAWN_RATE"]="32"
os.environ["SCRIPT"]="endpoint_user.py"
results_locust_path = os.path.join("output", "locust-testing")
os.environ["RESULTS_PREFIX"]=f"{results_locust_path}/results-{ts}"
    
try:
    with open("run_locust.log", "w") as logfile:
        print(f"Start Locust testing; logfile: run_locust.log; results: {results_locust_path}")
        path = os.path.join(os.getcwd(), "run_locust.sh")
        os.chmod(path, stat.S_IRUSR | stat.S_IEXEC)
        process = subprocess.Popen(path, encoding="utf-8", 
                                shell=True,stdout=logfile,stderr=subprocess.STDOUT)
        process.wait()
        logfile.flush()
        print(f"Locust testing completed")
except Exception as e:
    print(f"exception occurred: {e}")


## Visualize Locust Results

Below we first visualize the results of the Locust testing in a tabel. 

In [ ]:
import pandas as pd
from IPython.display import display
import numpy as np

results_path = os.environ["RESULTS_PREFIX"] + "_stats.csv"
df = pd.read_csv(results_path)
df = df.replace(np.nan, '')

top_n = 1
caption=f"Locust results"
df = df.truncate(after=top_n - 1, axis=0)
df = df.style \
      .format(precision=6) \
        .set_properties(**{'text-align': 'left'}) \
        .set_caption(caption)
display(df)

### Shutdown inference server

Next, we shutdown inference server.

In [ ]:
! {script_path} down